In [1]:
import pandas as pd
import numpy as np
import os

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pwd

'C:\\Users\\User\\Desktop\\Books\\Term 2\\AML 2404\\Code'

In [3]:
df = pd.read_excel('essay_set_questions.xlsx')
df.head(2)

,essay_set,type_of_essay,essay_type,grade_level,essay_prompt,essay_question,training_set_size,valid_set_size,test_set_size,min_domain1_score,max_domain1_score,min_domain2_score,max_domain2_score,has_domain1rater1,has_domain1rater2,has_domain1rater3,has_domain1_score,has_domain2rater1,has_domain2rater2,has_domain2_score
0,1,persuasive / narrative / expository,0,8,"More and more people use computers, but not ev...",Write a letter to your local newspaper in whic...,1783,589,594,2,12,NaN,NaN,1,1,0,1,0,0,0
1,2,persuasive / narrative / expository,0,10,"Censorship in the Libraries\n""All of us can th...",Write a persuasive essay to a newspaper reflec...,1800,600,600,1,6,1.0,4.0,1,1,0,1,1,1,1


In [4]:
import string
import re

punc_list = []
for char in string.punctuation:
    punc_list.append(char)
    
from nltk.corpus import stopwords
stopwordsEng = stopwords.words('English')
myOwn_stopwords = punc_list  + stopwordsEng

In [5]:
# Define functions required for processing text

def to_lower(text):
    text_lower = text.lower()
    return text_lower

def tokenize_text(text):
    text_tokens = word_tokenize(text)
    return text_tokens

def remove_stopwords(text):
    text_tokens_wo_stopwords = []
    for word in text:
        if not word in myOwn_stopwords:
            text_tokens_wo_stopwords.append(word)        
    return text_tokens_wo_stopwords




In [6]:
df['essay_prompt_keywords'] = df.essay_prompt.apply(lambda x: remove_stopwords(tokenize_text(to_lower(str(x)))))
df['essay_question_keywords'] = df.essay_question.apply(lambda x: remove_stopwords(tokenize_text(to_lower(str(x)))))
df.head(2)

,essay_set,type_of_essay,essay_type,grade_level,essay_prompt,essay_question,training_set_size,valid_set_size,test_set_size,min_domain1_score,...,max_domain2_score,has_domain1rater1,has_domain1rater2,has_domain1rater3,has_domain1_score,has_domain2rater1,has_domain2rater2,has_domain2_score,essay_prompt_keywords,essay_question_keywords
0,1,persuasive / narrative / expository,0,8,"More and more people use computers, but not ev...",Write a letter to your local newspaper in whic...,1783,589,594,2,...,NaN,1,1,0,1,0,0,0,"[people, use, computers, everyone, agrees, ben...","[write, letter, local, newspaper, state, opini..."
1,2,persuasive / narrative / expository,0,10,"Censorship in the Libraries\n""All of us can th...",Write a persuasive essay to a newspaper reflec...,1800,600,600,1,...,4.0,1,1,0,1,1,1,1,"[censorship, libraries, '', us, think, book, h...","[write, persuasive, essay, newspaper, reflecti..."


In [7]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

import selenium
from selenium import webdriver



In [8]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [9]:
def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.co.uk/search?q=" + query)

    links = list(itertools.islice(response.html.absolute_links, 10))
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')
    
    file_types = ('.pdf', '.txt')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)
        if url.endswith(file_types):
            links.remove(url)

    return links

In [10]:
import itertools
query = "data science blogs"
response = get_source("https://www.google.co.uk/search?q=" + query)
mylink = list(itertools.islice(response.html.absolute_links, 5))
print(mylink)

['https://www.google.co.uk/search?q=data+science+blogs&ei=HgmLYZicHpSdgQbRgJmYBQ&start=90&sa=N&ved=2ahUKEwiYo5afvIz0AhWUTsAKHVFABlMQ8tMDegQIARBM', 'https://www.google.co.uk/search?q=Artificial+intelligence&stick=H4sIAAAAAAAAAOOQUWLXz9U3yM2uMhJNSSxJVChOzkzNS05VKE9NKs4sSS2OEkIRTsrJTy8-xcirn65vaJhubpBiFF9YeYqRC2RIem5KilkhlGNinpORXgZXaZCdYmaUbv6LUdQFmy2LWMUdi0oy0zKTMxNzFDLzSlJzcjLTQUpusUky3JjJZZpxKdD1nWT83dd_N-X6pX04wDLpfzUAZMb1u74AAAA&sa=X&ved=2ahUKEwiYo5afvIz0AhWUTsAKHVFABlMQxA16BAgUEAc', 'https://support.google.com/websearch/answer/181196?hl=en-CA', 'https://www.google.co.uk/search?q=Data+science+news&sa=X&ved=2ahUKEwiYo5afvIz0AhWUTsAKHVFABlMQ1QJ6BAgPEAE', 'https://www.google.co.uk/search?q=How+do+I+write+a+data+science+blog%3F&sa=X&ved=2ahUKEwiYo5afvIz0AhWUTsAKHVFABlMQzmd6BAg0EAU']


In [11]:
scrape_google("data science blogs")

['https://blog.feedspot.com/data_science_blogs/',
 'https://www.dataquest.io/blog/']

In [14]:
df['essay_question_keywords'][0]

['write',
 'letter',
 'local',
 'newspaper',
 'state',
 'opinion',
 'effects',
 'computers',
 'people',
 'persuade',
 'readers',
 'agree']

In [15]:
df['essay_question_key_sent'] =  [' '.join(i) for i in df['essay_question_keywords']]
df['essay_prompt_key_sent'] =  [' '.join(i) for i in df['essay_prompt_keywords']]
df.head(2)

,essay_set,type_of_essay,essay_type,grade_level,essay_prompt,essay_question,training_set_size,valid_set_size,test_set_size,min_domain1_score,...,has_domain1rater2,has_domain1rater3,has_domain1_score,has_domain2rater1,has_domain2rater2,has_domain2_score,essay_prompt_keywords,essay_question_keywords,essay_question_key_sent,essay_prompt_key_sent
0,1,persuasive / narrative / expository,0,8,"More and more people use computers, but not ev...",Write a letter to your local newspaper in whic...,1783,589,594,2,...,1,0,1,0,0,0,"[people, use, computers, everyone, agrees, ben...","[write, letter, local, newspaper, state, opini...",write letter local newspaper state opinion eff...,people use computers everyone agrees benefits ...
1,2,persuasive / narrative / expository,0,10,"Censorship in the Libraries\n""All of us can th...",Write a persuasive essay to a newspaper reflec...,1800,600,600,1,...,1,0,1,1,1,1,"[censorship, libraries, '', us, think, book, h...","[write, persuasive, essay, newspaper, reflecti...",write persuasive essay newspaper reflecting vi...,censorship libraries '' us think book hope non...


In [16]:
df['ques_links'] = df['essay_question_key_sent'].apply(lambda x: scrape_google(str(x))) 
df['prompt_links'] = df['essay_prompt_key_sent'].apply(lambda x: scrape_google(str(x))) 
df.head(2)

,essay_set,type_of_essay,essay_type,grade_level,essay_prompt,essay_question,training_set_size,valid_set_size,test_set_size,min_domain1_score,...,has_domain1_score,has_domain2rater1,has_domain2rater2,has_domain2_score,essay_prompt_keywords,essay_question_keywords,essay_question_key_sent,essay_prompt_key_sent,ques_links,prompt_links
0,1,persuasive / narrative / expository,0,8,"More and more people use computers, but not ev...",Write a letter to your local newspaper in whic...,1783,589,594,2,...,1,0,0,0,"[people, use, computers, everyone, agrees, ben...","[write, letter, local, newspaper, state, opini...",write letter local newspaper state opinion eff...,people use computers everyone agrees benefits ...,[https://www.zerotothree.org/resources/480-how...,[http://www.ohrc.on.ca/en/policy-accessible-ed...
1,2,persuasive / narrative / expository,0,10,"Censorship in the Libraries\n""All of us can th...",Write a persuasive essay to a newspaper reflec...,1800,600,600,1,...,1,1,1,1,"[censorship, libraries, '', us, think, book, h...","[write, persuasive, essay, newspaper, reflecti...",write persuasive essay newspaper reflecting vi...,censorship libraries '' us think book hope non...,[https://www.webjunction.org/documents/webjunc...,[]


In [17]:
df['ques_links'][0]

['https://www.zerotothree.org/resources/480-how-to-write-a-letter-to-the-editor-and-an-opinion-editorial']

In [18]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_page_load_timeout(60)
print("Current session is {}".format(driver.session_id))
# driver.close()
# from selenium import webdriver
# driver = webdriver.Chrome()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.69/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\User\.wdm\drivers\chromedriver\win32\95.0.4638.69]


Current session is 7165be3ea1d54b8e665b43d43d86bfc5


In [15]:
# driver.get(url)
# doc = driver.page_source

In [19]:

link_contents = []

def get_web_text(link):
    
    for i in range(len(link)):
        
        try:            
            driver.get(link[i])            
            doc = driver.page_source
            text_content = driver.find_element_by_xpath("/html/body").text
            link_contents.append(text_content)
            print(driver.current_url)

        except:
#             isrunning = 0
#             print("Exception has been thrown. " + str(ex))
            driver.delete_all_cookies()
            print("Failed to Load:", driver.current_url)
            driver.close()
    
#         driver.get(link[i])
#         doc = driver.page_source
#         text_content = driver.find_element_by_xpath("/html/body").text
#         link_contents.append(text_content)
    
    return link_contents
    

In [20]:
df['ques_web_content'] = df['ques_links'].apply(lambda x: get_web_text(x))

https://www.zerotothree.org/resources/480-how-to-write-a-letter-to-the-editor-and-an-opinion-editorial
https://www.webjunction.org/documents/webjunction/Censorship_in_Schools_Learning_Speaking_and_Thinking_Freely_The_First_Amendment_in_Schools.html
https://usermanual.wiki/Pdf/Data20Set2023220Scoring20Guide.648402301/html
https://www.quora.com/Do-you-believe-that-certain-materials-such-as-books-music-movies-magazines-etc-should-be-removed-from-the-shelves-if-they-are-found-offensive
https://www.triand.com/bundles/167311-istep-2010-2012-la-eoc-grade-10
https://www.explainthatstuff.com/bicycles.html
https://www.sciencedirect.com/science/article/pii/S1369847818306934
https://www.bikeradar.com/advice/fitness-and-training/30-great-benefits-of-cycling/
https://www.bikeradar.com/advice/fitness-and-training/30-great-benefits-of-cycling/
http://englishiva1011.pbworks.com/f/HIBISCUS.PDF
https://www.coursehero.com/file/p1r7hsc/Where-you-headed-in-such-a-hurry-Yosemite-I-whispered-Know-the-best-way

In [21]:
df['prompt_web_content'] = df['prompt_links'].apply(lambda x: get_web_text(x))

http://www.ohrc.on.ca/en/policy-accessible-education-students-disabilities
http://www.csun.edu/~lic42878/computers.html
https://www.pewresearch.org/internet/2019/10/28/4-the-internet-will-continue-to-make-life-better/
https://www.pewresearch.org/internet/2019/10/28/4-the-internet-will-continue-to-make-life-better/
https://www.coursehero.com/file/p1r7hsc/Where-you-headed-in-such-a-hurry-Yosemite-I-whispered-Know-the-best-way-to-get/
https://en.wikipedia.org/wiki/Narciso_Rodriguez
https://arxiv.org/pdf/2005.12078.pdf
https://greatergood.berkeley.edu/article/item/four_reasons_to_cultivate_patience
https://contentmentquesting.com/10-powerful-benefits-of-being-patient/
https://www.merriam-webster.com/dictionary/patient
https://www.scientificamerican.com/article/whats-so-funny-the-science-of-why-we-laugh/
https://greatergood.berkeley.edu/article/item/how_laughter_brings_us_together


In [84]:
df['ques_web_content'][5]

["Homepage\nAccessibility links\nSkip to content\nAccessibility Help\nSign in\nNews\nSport\nReel\nWorklife\nTravel\nFuture\nMore\nSearch the BBC\nSearch the BBC\nBitesize\nChange language\nHome\nLearn\nSupport\nCareers\nMy Bitesize\nAll Bitesize\nKS3\nWriting to persuade, argue and advise\nWhen writing to argue, persuade and advise, you are putting forward your view to the reader. Each purpose has different techniques.\nPart ofEnglishWriting skills\nAdd to My Bitesize\nRevise\nvideo\nVideo\nquiz\nTest\nprevious\n1\n2\n3\n4\n5\nnext\nThe persuader's toolkit\nWhenever you are planning to write persuasively, always think carefully about some of the techniques that you could use to persuade your readers. Below is a checklist of tools, they are similar to the 'writing to argue techniques' but focus on persuasion.\nYou don’t need to use all of them but you should definitely use a range.\nAnecdotes - these are short accounts of a real event told in the form of a very brief story. Their effect

In [88]:
len(df['ques_web_content'])

8

In [91]:
df.head()

,essay_set,type_of_essay,essay_type,grade_level,essay_prompt,essay_question,training_set_size,valid_set_size,test_set_size,min_domain1_score,...,has_domain2_score,essay_prompt_keywords,essay_question_keywords,essay_question_key_sent,essay_prompt_key_sent,ques_links,prompt_links,ques_web_content,prompt_web_content,processed_ques_content
0,1,persuasive / narrative / expository,0,8,"More and more people use computers, but not ev...",Write a letter to your local newspaper in whic...,1783,589,594,2,...,0,"[people, use, computers, everyone, agrees, ben...","[write, letter, local, newspaper, state, opini...",write letter local newspaper state opinion eff...,people use computers everyone agrees benefits ...,[https://www.bbc.co.uk/bitesize/guides/zyydjxs...,[https://www.pewresearch.org/internet/2018/07/...,[Homepage\nAccessibility links\nSkip to conten...,[Homepage\nAccessibility links\nSkip to conten...,"[[homepage, accessibility, links, skip, conten..."
1,2,persuasive / narrative / expository,0,10,"Censorship in the Libraries\n""All of us can th...",Write a persuasive essay to a newspaper reflec...,1800,600,600,1,...,1,"[censorship, libraries, '', us, think, book, h...","[write, persuasive, essay, newspaper, reflecti...",write persuasive essay newspaper reflecting vi...,censorship libraries '' us think book hope non...,[https://www.quora.com/Do-you-believe-that-cer...,[https://www.sonyasones.com/FAQs/bannedbooks.h...,[Homepage\nAccessibility links\nSkip to conten...,[Homepage\nAccessibility links\nSkip to conten...,"[[homepage, accessibility, links, skip, conten..."
2,3,source dependent responses,1,10,ROUGH ROAD AHEAD: Do Not Exceed Posted Speed L...,Write a response that explains how the feature...,1726,568,564,0,...,0,"[rough, road, ahead, exceed, posted, speed, li...","[write, response, explains, features, setting,...",write response explains features setting affec...,rough road ahead exceed posted speed limit joe...,[https://books.google.ca/books?id=FfKhidPNKIkC...,[],[Homepage\nAccessibility links\nSkip to conten...,[Homepage\nAccessibility links\nSkip to conten...,"[[homepage, accessibility, links, skip, conten..."
3,4,source dependent responses,1,10,"Winter Hibiscus by Minfong Ho\nSaeng, a teenag...","Read the last paragraph of the story.\n\n""When...",1772,586,590,0,...,0,"[winter, hibiscus, minfong, ho, saeng, teenage...","[read, last, paragraph, story, ``, come, back,...",read last paragraph story `` come back saeng v...,winter hibiscus minfong ho saeng teenage girl ...,[],[https://elsyoksendiri.wordpress.com/2021/06/1...,[Homepage\nAccessibility links\nSkip to conten...,[Homepage\nAccessibility links\nSkip to conten...,"[[homepage, accessibility, links, skip, conten..."
4,5,source dependent responses,1,8,Narciso Rodriguez\nfrom Home: The Blueprints o...,Describe the mood created by the author in the...,1805,601,600,0,...,0,"[narciso, rodriguez, home, blueprints, lives, ...","[describe, mood, created, author, memoir, supp...",describe mood created author memoir support an...,narciso rodriguez home blueprints lives parent...,[https://www.louisianabelieves.com/docs/defaul...,[],[Homepage\nAccessibility links\nSkip to conten...,[Homepage\nAccessibility links\nSkip to conten...,"[[homepage, accessibility, links, skip, conten..."


In [83]:
df.to_csv("Web_Scraped_data.csv")

In [89]:
df['processed_ques_content'] = df['ques_web_content'].apply(lambda x: [remove_stopwords(tokenize_text(to_lower(str(i)))) 
                                                                 for i in x])


In [90]:
df.to_csv("Web_Scraped_data_processed.csv")

In [92]:
df['processed_prompt_content'] = df['prompt_web_content'].apply(lambda x: [remove_stopwords(tokenize_text(to_lower(str(i)))) 
                                                                 for i in x])

## Read the essays file

In [94]:
df_train = pd.read_csv("essay_training_data_processed.csv")

In [99]:
df_train.columns

Index(['Unnamed: 0', 'essay_id', 'essay_set', 'essay', 'domain1_score',
       'number_of_words', 'number_of_sentences', 'error_score',
       'error_score_sent%', 'error_score_word%'],
      dtype='object')

In [95]:
df.columns

Index(['essay_set', 'type_of_essay', 'essay_type', 'grade_level',
       'essay_prompt', 'essay_question', 'training_set_size', 'valid_set_size',
       'test_set_size', 'min_domain1_score', 'max_domain1_score',
       'min_domain2_score', 'max_domain2_score', 'has_domain1rater1',
       'has_domain1rater2', 'has_domain1rater3', 'has_domain1_score',
       'has_domain2rater1', 'has_domain2rater2', 'has_domain2_score',
       'essay_prompt_keywords', 'essay_question_keywords',
       'essay_question_key_sent', 'essay_prompt_key_sent', 'ques_links',
       'prompt_links', 'ques_web_content', 'prompt_web_content',
       'processed_ques_content', 'processed_prompt_content'],
      dtype='object')

In [98]:
df_text = df[['essay_set', 'essay_prompt', 'essay_question' ,'processed_ques_content', 'processed_prompt_content']]

In [102]:
df_new = df_train.merge(df_text, how = 'inner', on = 'essay_set' )

In [103]:
df_new.columns

Index(['Unnamed: 0', 'essay_id', 'essay_set', 'essay', 'domain1_score',
       'number_of_words', 'number_of_sentences', 'error_score',
       'error_score_sent%', 'error_score_word%', 'essay_prompt',
       'essay_question', 'processed_ques_content', 'processed_prompt_content'],
      dtype='object')

In [104]:
df_new.to_csv("Web_Scraped_essay_answers.csv")

In [105]:
df_new.head()

,Unnamed: 0,essay_id,essay_set,essay,domain1_score,number_of_words,number_of_sentences,error_score,error_score_sent%,error_score_word%,essay_prompt,essay_question,processed_ques_content,processed_prompt_content
0,3289,4484,2,Do I think that certain materials that are off...,0.666667,459,27,18,0.667,0.039,"Censorship in the Libraries\n""All of us can th...",Write a persuasive essay to a newspaper reflec...,"[[homepage, accessibility, links, skip, conten...","[[homepage, accessibility, links, skip, conten..."
1,1804,2999,2,"I think that books, magazines, papers etc shou...",0.500000,337,19,23,1.211,0.068,"Censorship in the Libraries\n""All of us can th...",Write a persuasive essay to a newspaper reflec...,"[[homepage, accessibility, links, skip, conten...","[[homepage, accessibility, links, skip, conten..."
2,2080,3275,2,"Dear @CAPS1 @CAPS2 Newspaper, or to whom this ...",0.500000,362,18,15,0.833,0.041,"Censorship in the Libraries\n""All of us can th...",Write a persuasive essay to a newspaper reflec...,"[[homepage, accessibility, links, skip, conten...","[[homepage, accessibility, links, skip, conten..."
3,2670,3865,2,"Katherine Paterson, who is an author, thought ...",0.666667,385,21,2,0.095,0.005,"Censorship in the Libraries\n""All of us can th...",Write a persuasive essay to a newspaper reflec...,"[[homepage, accessibility, links, skip, conten...","[[homepage, accessibility, links, skip, conten..."
4,3267,4462,2,I believe censorship is necessary. If all of t...,0.666667,815,32,16,0.500,0.020,"Censorship in the Libraries\n""All of us can th...",Write a persuasive essay to a newspaper reflec...,"[[homepage, accessibility, links, skip, conten...","[[homepage, accessibility, links, skip, conten..."


In [ ]:
url = df.ques_links[0]

In [ ]:
!pip install webdriver-manager

https://stackoverflow.com/questions/29858752/error-message-chromedriver-executable-needs-to-be-available-in-the-path/52878725#52878725

In [ ]:
# #open text file
# text_file = open("Webpage_contents.txt", "w")
 
# #write string to file
# text_file.write(doc)
 
# #close file
# text_file.close()


In [ ]:
print(driver.find_element_by_xpath("/html/body").text)

In [ ]:
print(results)

In [ ]:
prompt_links = scrape_google(df['essay_prompt_key_sent'][0])
prompt_links

In [ ]:
# For functions to preprocess texts

# Get vectors  from text using Tfidf
def get_vectors(*str):
    text = [t for t in str]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer.fit(text)
    
    
    myVectors = tfidf_vectorizer.transform(text).toarray()
    return myVectors

# Get cosine similarity
def get_cosine_similarity(received_vectors):
    return cosine_similarity(received_vectors)

In [ ]:
cos_sim = get_cosine_similarity(vectors)